In [1]:
import numpy as np
import matplotlib
import os
import cv2
from skimage import io
from skimage import color, exposure, transform
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter, ImageStat, ImageEnhance 
import imutils
import argparse
import ntpath
import sys
import matplotlib.pyplot as plt
import glob
import math
from blend_modes import blend_modes
import shutil
from datetime import datetime
import random

In [2]:
def load_paths(directory):
    paths = []
    for files in os.listdir(directory):
        if (files != ".DS_Store"):
            paths.append(directory+'/'+files)
    return paths

In [3]:
def find_borders(img,pix):
    borders = []    
    for y in range(0,img.size[1]):
        found = False
        has_grey = False
        
        for x in range(1,img.size[0]):
            
            r = pix[x,y][0]
            g = pix[x,y][1]
            b = pix[x,y][2]
                
            prev_r = pix[x-1,y][0]
            prev_g = pix[x-1,y][1]
            prev_b = pix[x-1,y][2]
            
            r_g = abs(r-g)
            r_b = abs(r-b)
            g_b = abs(g-b)
            
            if (r_g<=15 and r_b<=15 and g_b<=15):
                has_grey = True

                
            if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    
                for i in range(0,x):
                    borders.append([i,y])
                found = True
                break   
            
        if ((not found)and(has_grey)):
            for i in range(0,img.size[0]-1):
                borders.append([i,y])  
              
        for x in range(img.size[0]-1,1,-1):
                
            r = pix[x,y][0]
            g = pix[x,y][1]
            b = pix[x,y][2]
                
            prev_r = pix[x-1,y][0]
            prev_g = pix[x-1,y][1]
            prev_b = pix[x-1,y][2]
                
            if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    
                for i in range(x,img.size[0]-1):
                    borders.append([i,y])
                break
                    
    return borders

In [4]:
def manipulate_images(paths):
    
    for image_path in paths:
        
        
        img = Image.open(image_path)
        img = img.convert('RGB')
        pix = img.load()
        
        borders = find_borders(img,pix)
        
        #img.save(image_path)
         
        
        image = cv2.imread(image_path, -1)
        
        
        channels = cv2.split(image)
        
        alpha_channel = np.ones(channels[0].shape, dtype=channels[0].dtype) * 255
        image_RGBA = cv2.merge((channels[0], channels[1], channels[2], alpha_channel))
        
        height, width, channels = image.shape
        
        #Deleting white perimeter of shape
        for i in range(0,img.size[0]-1):
            image_RGBA[0,i][3] = 0
            image_RGBA[img.size[1]-1,i][3] = 0
        
        for i in range(0,img.size[1]-1):
            image_RGBA[i,0][3] = 0
            image_RGBA[i,img.size[0]-1][3] = 0
            
        
        
        for border in borders:
            image_RGBA[border[1],border[0]][3] = 0
            
        
        head, tail = ntpath.split(image_path)
        
        title,extension = tail.split('.')
        cv2.imwrite("Traffic_Signs_Templates/Processed_Images/"+title+".png", image_RGBA)
        img.close()
        
        
        

In [5]:
directory = 'Traffic_Signs_Templates/Images'
if (not os.path.exists("Traffic_Signs_Templates/Processed_Images")):
    os.mkdir("Traffic_Signs_Templates/Processed_Images")
paths = load_paths(directory)
manipulate_images(paths)

In [6]:
def img_transform(paths):
    
    for image_path in paths:
        
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        rows,cols,ch = img.shape
        t = []
        for i in range(0,100):
            t.append(i)
            
        #FORWARD FACING
        dst = img
        
        #EAST FACING
        pts1 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols/10,rows/2]])
        pts2 = np.float32([[cols/5,rows/5],[cols/2,rows/8],[cols/5,rows/1.8]])
        M = cv2.getAffineTransform(pts1,pts2)
        dst1 = cv2.warpAffine(img,M,(cols,rows))
        
        #NORTH-WEST FACING
        pts3 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts4 = np.float32([[cols*4.5/5,rows/5],[cols/2,rows/8],[cols*4.5/5,rows/1.8]])
        M = cv2.getAffineTransform(pts3,pts4)
        dst2 = cv2.warpAffine(img,M,(cols,rows))
        
        #LEFT TILTED FORWARD FACING
        pts5 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols/10,rows/2]])
        pts6 = np.float32([[cols/12,rows/6],[cols/2.1,rows/8],[cols/10,rows/1.8]])
        M = cv2.getAffineTransform(pts5,pts6)
        dst3 = cv2.warpAffine(img,M,(cols,rows))
        
        #RIGHT TILTED FORWARD FACING
        pts7 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts8 = np.float32([[cols*10/12,rows/6],[cols/2.2,rows/8],[cols*8.4/10,rows/1.8]])
        M = cv2.getAffineTransform(pts7,pts8)
        dst4 = cv2.warpAffine(img,M,(cols,rows))
        
        #WEST FACING
        pts9 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts10 = np.float32([[cols/9.95,rows/10],[cols/2.05,rows/9.95],[cols*9/10,rows/2.05]])
        M = cv2.getAffineTransform(pts9,pts10)
        dst5 = cv2.warpAffine(img,M,(cols,rows))
        
        #RIGHT TILTED FORWARD FACING
        pts11 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts12 = np.float32([[cols*9/10,rows/10],[cols/2,rows/9],[cols*8.95/10,rows/2.05]])
        M = cv2.getAffineTransform(pts11,pts12)
        dst6 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION
        pts13 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts14 = np.float32([[cols/9.8,rows/9.8],[cols/2,rows/9.8],[cols*8.8/10,rows/2.05]])
        M = cv2.getAffineTransform(pts13,pts14)
        dst7 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 2
        pts15 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts16 = np.float32([[cols/11,rows/10],[cols/2.1,rows/10],[cols*8.5/10,rows/1.95]])
        M = cv2.getAffineTransform(pts15,pts16)
        dst8 = cv2.warpAffine(img,M,(cols,rows))
        
        
        #FORWARD FACING W/ DISTORTION 3
        pts17 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts18 = np.float32([[cols/11,rows/11],[cols/2.1,rows/10],[cols*10/11,rows/1.95]])
        M = cv2.getAffineTransform(pts17,pts18)
        dst9 = cv2.warpAffine(img,M,(cols,rows))
        
        
        #FORWARD FACING W/ DISTORTION 4
        pts19 = np.float32([[cols*9.5/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts20 = np.float32([[cols*9.35/10,rows/9.99],[cols/2.05,rows/9.95],[cols*9.05/10,rows/2.03]])
        M = cv2.getAffineTransform(pts19,pts20)
        dst10 = cv2.warpAffine(img,M,(cols,rows))
         
        #FORWARD FACING W/ DISTORTION 5
        pts21 = np.float32([[cols*9.5/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts22 = np.float32([[cols*9.65/10,rows/9.95],[cols/1.95,rows/9.95],[cols*9.1/10,rows/2.02]])
        M = cv2.getAffineTransform(pts21,pts22)
        dst11 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 6
        pts23 = np.float32([[cols*9.25/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts24 = np.float32([[cols*9.55/10,rows/9.85],[cols/1.9,rows/10],[cols*9.3/10,rows/2.04]])
        M = cv2.getAffineTransform(pts23,pts24)
        dst12 = cv2.warpAffine(img,M,(cols,rows))
        
        #SHRINK 1
        pts25 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts26 = np.float32([[cols*8/10,rows/10],[cols*1.34/3,rows/10.5],[cols*8.24/10,rows/2.5]])
        M = cv2.getAffineTransform(pts25,pts26)
        dst13 = cv2.warpAffine(img,M,(cols,rows))
        
        #SHRINK 2
        pts27 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts28 = np.float32([[cols*8.5/10,rows*3.1/10],[cols/2,rows*3/10],[cols*8.44/10,rows*1.55/2.5]])
        M = cv2.getAffineTransform(pts27,pts28)
        dst14 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 7
        pts29 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts30 = np.float32([[cols*8.85/10,rows/9.3],[cols/1.9,rows/10.5],[cols*8.8/10,rows/2.11]])
        M = cv2.getAffineTransform(pts29,pts30)
        dst15 = cv2.warpAffine(img,M,(cols,rows))
        
        
        #FORWARD FACING W/ DISTORTION 8
        pts31 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts32 = np.float32([[cols*8.75/10,rows/9.1],[cols/1.95,rows/8],[cols*8.5/10,rows/2.05]])
        M = cv2.getAffineTransform(pts31,pts32)
        dst16 = cv2.warpAffine(img,M,(cols,rows))
        
        
        #FORWARD FACING W/ DISTORTION 9
        pts33 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts34 = np.float32([[cols*8.75/10,rows/9.1],[cols/1.95,rows/9],[cols*8.5/10,rows/2.2]])
        M = cv2.getAffineTransform(pts33,pts34)
        dst17 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 10
        pts35 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts36 = np.float32([[cols*8.75/10,rows/8],[cols/1.95,rows/8],[cols*8.75/10,rows/2]])
        M = cv2.getAffineTransform(pts35,pts36)
        dst18 = cv2.warpAffine(img,M,(cols,rows))
        
        
        #FORWARD FACING W/ DISTORTION 11
        pts37 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts38 = np.float32([[cols*8.8/10,rows/7],[cols/1.95,rows/7],[cols*8.8/10,rows/2]])
        M = cv2.getAffineTransform(pts37,pts38)
        dst19 = cv2.warpAffine(img,M,(cols,rows))
        
        head, tail = ntpath.split(image_path)
        
        title,extension = tail.split('.')
        
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[0])+".png",dst)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[1])+".png",dst1)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[2])+".png",dst2)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[3])+".png",dst3)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[4])+".png",dst4)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[5])+".png",dst5)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[6])+".png",dst6)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[7])+".png",dst7)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[8])+".png",dst8)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[9])+".png",dst9)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[10])+".png",dst10)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[11])+".png",dst11)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[12])+".png",dst12)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[13])+".png",dst13)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[14])+".png",dst14)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[15])+".png",dst15)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[16])+".png",dst16)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[17])+".png",dst17)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[18])+".png",dst18)
        cv2.imwrite("Traffic_Signs_Templates/Transformed_Images/"+title+"/"+str(t[19])+".png",dst19)
        
        plt.show()
        
        

In [7]:
directory = 'Traffic_Signs_Templates/Processed_Images'
if (not os.path.exists("Traffic_Signs_Templates/Transformed_Images")):
    for path in paths:
        head, tail = ntpath.split(path)    
        title,extension = tail.split('.')
        os.makedirs("Traffic_Signs_Templates/Transformed_Images/"+title)
paths = load_paths(directory)
img_transform(paths)

In [8]:
def find_image_exposure(paths,channels):
    
    exposures = []
    
    for image_path in paths:
        img = Image.open(image_path)
        im = Image.open(image_path).convert('LA')
        
        stat = ImageStat.Stat(im)
        
        #Average pixel brighness
        avg = stat.mean[0]
        
        #RMS pixel brighness
        rms = stat.rms[0]
        
        stat2 = ImageStat.Stat(img)
        
        #Consider the number of channels
        #background may have RGB while traffic sign has RGBA
        if (channels==3):
            #Average pixels preceived brightness
            r,g,b = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)
        else:
            #Average pixels preceived brightness
            r,g,b,a = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)
             

    return exposures     

In [9]:
def to_png(directory):
    for files in load_paths(directory):
        title,extension = files.split('.')
        img = Image.open(files).convert('RGBA')
        if (not extension == "png"):
            os.remove(files)
        img.save(title+".png")

In [11]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

to_png("Google_search_backgrounds/UK_urban")
to_png("Google_search_backgrounds/UK_rural")

In [12]:
def exposure_manipulation(signs_paths,backgrounds_paths):
    
    background_exposures = find_image_exposure(background_paths,4)
    signs_exposures = find_image_exposure(signs_paths,4)
    
    for i in range(0,len(background_paths)):
        
        print "Processed: "+str(float(i)/float(len(background_paths))*100)+" %"
        
        img = Image.open(background_exposures[i][0])


        for sign_path in signs_paths:

            dirc,sub,el = background_exposures[i][0].split('/')
            title,extension = el.split('.')

            parent_dir,sub_dir,folder,element = sign_path.split('/')
            head,tail = element.split('.')

            brightness_avrg = 1.0
            brightness_rms = 1.0
            brightness_avrg_perceived = 1.0
            brightness_rms_perceived = 1.0
            brightness_avrg2 = 1.0
            brightness_rms2 = 1.0
            
            # abs(desired_brightness - actual_brightness)/ abs(brightness_float_value) = ratio
            avrg_ratio = 11.0159464507

            rms_ratio = 8.30320014372

            percieved_avrg_ratio = 3.85546373056

            percieved_rms_ratio = 35.6344530649

            avrg2_ratio = 1.20354549572

            rms2_ratio = 40.1209106864

            peak = Image.open(sign_path).convert('LA')
            peak2 = Image.open(sign_path).convert('RGBA')

            stat = ImageStat.Stat(peak)
            avrg = stat.mean[0]
            rms = stat.rms[0]

            #IMAGE MANIPULATION MAIN CODE STARTS
            

            #MINIMISE MARGIN BASED ON AVERAGE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(avrg-float(background_exposures[i][1]))
            
            brightness_avrg = margin/avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright = enhancer.enhance(brightness_avrg)
            stat = ImageStat.Stat(avrg_bright)
            avrg = stat.mean[0]
            

            
            #MINIMISE MARGIN BASED ON ROOT MEAN SQUARE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(rms-float(background_exposures[i][2]))

            brightness_rms = margin/rms_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright = enhancer.enhance(brightness_rms)
            stat = ImageStat.Stat(rms_bright)
            rms = stat.rms[0]
            
            



            
            #MINIMISE MARGIN BASED ON AVERAGE FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            stat2 = ImageStat.Stat(peak2)
            r,g,b,a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            margin = abs(avrg_perceived-float(background_exposures[i][3]))
            
            brightness_avrg_perceived = margin/percieved_avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright_perceived = enhancer.enhance(brightness_avrg_perceived)
            stat2 = ImageStat.Stat(avrg_bright_perceived)
            r,g,b,a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))        
            



            #MINIMISE MARGIN BASED ON RMS FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            margin = abs(rms_perceived-float(background_exposures[i][4]))

            brightness_rms_perceived = margin/percieved_rms_ratio 

            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright_perceived = enhancer.enhance(brightness_rms_perceived)
            stat2 = ImageStat.Stat(rms_bright_perceived)
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))        
            
            





            stat3 = ImageStat.Stat(peak2)
            avrg2 = stat3.mean[0]
            rms2 = stat3.rms[0]

            """
            #FUSION OF THE TWO AVERAGING METHODS
            margin = abs(avrg2-float(background_exposures[i][1]))
            brightness_avrg2 = margin/avrg2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright2 = enhancer.enhance(brightness_avrg2)
            stat3 = ImageStat.Stat(avrg_bright2)
            avrg2 = stat3.mean[0]       
            """
            


            """
            #FUSION OF THE TWO RMS METHODS
            margin = abs(rms2-float(background_exposures[i][2]))
            brightness_rms2 = margin/rms2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright2 = enhancer.enhance(brightness_rms2)
            stat3 = ImageStat.Stat(rms_bright2)
            rms2 = stat3.rms[0]
            """
            
            avrg_bright = avrg_bright.resize((150,150), Image.ANTIALIAS)
            rms_bright = rms_bright.resize((150,150), Image.ANTIALIAS)
            avrg_bright_perceived = avrg_bright_perceived.resize((150,150), Image.ANTIALIAS)
            rms_bright_perceived = rms_bright_perceived.resize((150,150), Image.ANTIALIAS)
            #avrg_bright2 = avrg_bright2.resize((150,150), Image.ANTIALIAS)
            #rms_bright2 = rms_bright2.resize((150,150), Image.ANTIALIAS)

            
            avrg_bright.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_AVERAGE."+tail)
            rms_bright.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_RMS."+tail)
            avrg_bright_perceived.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_AVERAGE_PERCEIVED."+tail)
            rms_bright_perceived.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_RMS_PERCEIVED."+tail)
            #avrg_bright2.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_AVERAGE2."+tail)
            #rms_bright2.save("Traffic_Signs_exposure_manipulation/"+sub+"/"+title+"/SIGN_"+folder+"/"+head+"_RMS2."+tail)
    print "Processed: "+str(100)+" %"
    print "Process was successful"

In [13]:
bg_dir = "Google_search_backgrounds"

for dirs in load_paths(bg_dir):    
    initial,subd = dirs.split('/')
    
    for background in load_paths(dirs):
        initial,subd,element = background.split('/')
        title,extension = element.split('.')
        
        for signp in load_paths("Traffic_Signs_Templates/Transformed_Images"):
            for sign in load_paths(signp):
                d,s,f,e = sign.split('/')
                head,tail = e.split('.')
            
                if (not os.path.exists("Traffic_Signs_exposure_manipulation/"+subd+"/"+title+"/SIGN_"+f)):
                    os.makedirs("Traffic_Signs_exposure_manipulation/"+subd+"/"+title+"/SIGN_"+f)
            

signs_paths = []
for p in load_paths("Traffic_Signs_Templates/Transformed_Images"):
    signs_paths = signs_paths + load_paths(p)

background_paths = load_paths("Google_search_backgrounds/UK_urban")
exposure_manipulation(signs_paths,background_paths)

Processed: 0.0 %
Processed: 0.2 %
Processed: 0.4 %
Processed: 0.6 %
Processed: 0.8 %
Processed: 1.0 %
Processed: 1.2 %
Processed: 1.4 %
Processed: 1.6 %
Processed: 1.8 %
Processed: 2.0 %
Processed: 2.2 %
Processed: 2.4 %
Processed: 2.6 %
Processed: 2.8 %
Processed: 3.0 %
Processed: 3.2 %
Processed: 3.4 %
Processed: 3.6 %
Processed: 3.8 %
Processed: 4.0 %
Processed: 4.2 %
Processed: 4.4 %
Processed: 4.6 %
Processed: 4.8 %
Processed: 5.0 %
Processed: 5.2 %
Processed: 5.4 %
Processed: 5.6 %
Processed: 5.8 %
Processed: 6.0 %
Processed: 6.2 %
Processed: 6.4 %
Processed: 6.6 %
Processed: 6.8 %
Processed: 7.0 %
Processed: 7.2 %
Processed: 7.4 %
Processed: 7.6 %
Processed: 7.8 %
Processed: 8.0 %
Processed: 8.2 %
Processed: 8.4 %
Processed: 8.6 %
Processed: 8.8 %
Processed: 9.0 %
Processed: 9.2 %
Processed: 9.4 %
Processed: 9.6 %
Processed: 9.8 %
Processed: 10.0 %
Processed: 10.2 %
Processed: 10.4 %
Processed: 10.6 %
Processed: 10.8 %
Processed: 11.0 %
Processed: 11.2 %
Processed: 11.4 %
Proces

Processed: 91.6 %
Processed: 91.8 %
Processed: 92.0 %
Processed: 92.2 %
Processed: 92.4 %
Processed: 92.6 %
Processed: 92.8 %
Processed: 93.0 %
Processed: 93.2 %
Processed: 93.4 %
Processed: 93.6 %
Processed: 93.8 %
Processed: 94.0 %
Processed: 94.2 %
Processed: 94.4 %
Processed: 94.6 %
Processed: 94.8 %
Processed: 95.0 %
Processed: 95.2 %
Processed: 95.4 %
Processed: 95.6 %
Processed: 95.8 %
Processed: 96.0 %
Processed: 96.2 %
Processed: 96.4 %
Processed: 96.6 %
Processed: 96.8 %
Processed: 97.0 %
Processed: 97.2 %
Processed: 97.4 %
Processed: 97.6 %
Processed: 97.8 %
Processed: 98.0 %
Processed: 98.2 %
Processed: 98.4 %
Processed: 98.6 %
Processed: 98.8 %
Processed: 99.0 %
Processed: 99.2 %
Processed: 99.4 %
Processed: 99.6 %
Processed: 99.8 %
Processed: 100 %
Process was successful


In [14]:
background_paths = load_paths("Google_search_backgrounds/UK_rural")
exposure_manipulation(signs_paths,background_paths)

Processed: 0.0 %
Processed: 0.2 %
Processed: 0.4 %
Processed: 0.6 %
Processed: 0.8 %
Processed: 1.0 %
Processed: 1.2 %
Processed: 1.4 %
Processed: 1.6 %
Processed: 1.8 %
Processed: 2.0 %
Processed: 2.2 %
Processed: 2.4 %
Processed: 2.6 %
Processed: 2.8 %
Processed: 3.0 %
Processed: 3.2 %
Processed: 3.4 %
Processed: 3.6 %
Processed: 3.8 %
Processed: 4.0 %
Processed: 4.2 %
Processed: 4.4 %
Processed: 4.6 %
Processed: 4.8 %
Processed: 5.0 %
Processed: 5.2 %
Processed: 5.4 %
Processed: 5.6 %
Processed: 5.8 %
Processed: 6.0 %
Processed: 6.2 %
Processed: 6.4 %
Processed: 6.6 %
Processed: 6.8 %
Processed: 7.0 %
Processed: 7.2 %
Processed: 7.4 %
Processed: 7.6 %
Processed: 7.8 %
Processed: 8.0 %
Processed: 8.2 %
Processed: 8.4 %
Processed: 8.6 %
Processed: 8.8 %
Processed: 9.0 %
Processed: 9.2 %
Processed: 9.4 %
Processed: 9.6 %
Processed: 9.8 %
Processed: 10.0 %
Processed: 10.2 %
Processed: 10.4 %
Processed: 10.6 %
Processed: 10.8 %
Processed: 11.0 %
Processed: 11.2 %
Processed: 11.4 %
Proces

Processed: 91.6 %
Processed: 91.8 %
Processed: 92.0 %
Processed: 92.2 %
Processed: 92.4 %
Processed: 92.6 %
Processed: 92.8 %
Processed: 93.0 %
Processed: 93.2 %
Processed: 93.4 %
Processed: 93.6 %
Processed: 93.8 %
Processed: 94.0 %
Processed: 94.2 %
Processed: 94.4 %
Processed: 94.6 %
Processed: 94.8 %
Processed: 95.0 %
Processed: 95.2 %
Processed: 95.4 %
Processed: 95.6 %
Processed: 95.8 %
Processed: 96.0 %
Processed: 96.2 %
Processed: 96.4 %
Processed: 96.6 %
Processed: 96.8 %
Processed: 97.0 %
Processed: 97.2 %
Processed: 97.4 %
Processed: 97.6 %
Processed: 97.8 %
Processed: 98.0 %
Processed: 98.2 %
Processed: 98.4 %
Processed: 98.6 %
Processed: 98.8 %
Processed: 99.0 %
Processed: 99.2 %
Processed: 99.4 %
Processed: 99.6 %
Processed: 99.8 %
Processed: 100 %
Process was successful


In [15]:
def avrg_pixel_rgb(image,chanels):
    stat = ImageStat.Stat(image)
    if (chanels == 4):
        r,g,b,a = stat.rms
    else:
        r,g,b = stat.rms
    
    return [r,g,b]

In [16]:
def find_bw_images(directory):
    images = []
    for signs in load_paths(directory):
        img = Image.open(signs).convert('RGBA')
        rgb = avrg_pixel_rgb(img,4)
        rg = abs(rgb[0]-rgb[1])
        rb = abs(rgb[0]-rgb[2])
        gb = abs(rgb[1]-rgb[2])
        
        temp = signs.split('/')
        head,tail = temp[-1].split('.')
                
        if (rg<=1 and rb<=1 and gb<=1):
            images.append(head)
    return images

In [17]:
def find_useful_signs(directory):
    bw_images = find_bw_images("Traffic_Signs_Templates/Images")
    for background_dir in load_paths(directory):
        
        for signs in load_paths(background_dir):
            temp = []
            for imgs in load_paths(signs):
                temp.append(imgs)
            exposures = find_image_exposure(temp,4)
            i = 0
            for images in load_paths(signs):
                
                
                
                #Find brightness
                img = Image.open(images).convert('RGBA')




                rgb = avrg_pixel_rgb(img,4)
                rg = abs(rgb[0]-rgb[1])
                rb = abs(rgb[0]-rgb[2])
                gb = abs(rgb[1]-rgb[2])
                
                    
                is_bw = False
                
                for s in bw_images:
                    if s in exposures[i][0]:
                        is_bw = True
                    
                if (rg<=16 and rb<=16 and gb<=16):
                    if (not is_bw):
                        os.remove(images)
                    #Threshold values for black and white images
                    elif (rgb[0]<70 and rgb[1]<70 and rgb[2]<70):
                        os.remove(images)
                    elif (rgb[0]>155 and rgb[1]>155 and rgb[2]>155):
                        os.remove(images)
                        
                elif (not is_bw):
                    #Delete light blue images
                    if(rgb[2]>rgb[0] and rgb[2]>=rgb[1]):
                        if (gb<=10):
                            os.remove(images)
                i = i+1
            

In [18]:
directory= "Traffic_Signs_exposure_manipulation/UK_urban"
find_useful_signs(directory)

In [19]:
directory= "Traffic_Signs_exposure_manipulation/UK_rural"
find_useful_signs(directory)

In [20]:
def insert_poisson_noise (image):
    vals = len(np.unique(image))
    vals = 2.05 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy

In [21]:
def insert_Gaussian_noise (image):
    row,col,ch= image.shape
    mean = 0
    var = 0.5
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [22]:
def insert_speckle_noise (image):
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy = image + image * gauss
    return noisy

In [23]:
def random_noise_method (image):
    """
    i = random.randint(1, 3)
    if (i == 1):
        return insert_poisson_noise(image)
    elif (i==2):
        return insert_Gaussian_noise(image)
    else:
        return insert_speckle_noise(image)
    """
    image.setflags(write=1)
    #Add noise in every pixel w/ random probability 0.4
    for im in image:
        px = 0
        for pixel in im:
            apply_noise = random.randint(0,100)
            #if random probability
            if apply_noise > 40:
                #RGB values
                R = pixel[0]
                G = pixel[1]
                B = pixel[2]
                A = pixel[3]
                #find current relative lumination for brighness
                #based on: https://en.wikipedia.org/wiki/Relative_luminance
                relative_lumination = 0.2126*R + 0.7152*G + 0.0722*B
                #find differences between RGB values     
                R_to_G = float(R)/float(G)
                RG = False
                if (R_to_G >= 1): RG=True
                R_to_B = float(R)/float(B)
                RB = False
                if (R_to_B >= 1): RB=True
                G_to_B = float(G)/float(B)
                GB = False
                if (G_to_B >= 1): GB=True
                equal = False
                if (R==G==B):equal==True

                #In order to determine the margin in which the new brighness
                #should be within, the upper and lower limits need to be foun
                #The Relative luminance in colorimetric spaces has normilised
                #values between 0 and 255
                upper_limit = 255
                lower_limit = 0
                if (relative_lumination + 40 < 255):
                    upper_limit = relative_lumination + 40
                if (relative_lumination - 40 > 0):
                    lower_limit = relative_lumination - 40

                #Compute new brighness value
                new_lumination = random.randint(int(lower_limit),int(upper_limit))

                #find the three possible solutions that satisfy
                #->The new lumination chosen based on the Relative luminance equation
                #->The precentages computed between every RGB value

                solutions = []

                for r in range(1,255):
                    for g in range(1,255):
                        for b in range(1,255):
                            r_to_g = float(r)/float(g)
                            rg = False
                            if (r_to_g >= 1): rg=True
                            r_to_b = float(r)/float(b)
                            rb = False
                            if (r_to_b >= 1): rb=True
                            g_to_b = float(g)/float(b)
                            gb = False
                            if (g_to_b >= 1): gb=True
                            e = False
                            if(r==g==b):
                                e=True
                            if (0.2126*r + 0.7152*g + 0.0722*b == 100) and rg==RG and rb==RB and gb==GB and e==equal:
                                solutions.append([r,g,b])

                #Find the solution that precentage wise is closer to the original
                #difference between the values
                percentages = []

                for solution in solutions:
                    r = solution[0]
                    g = solution[1]
                    b = solution[2]
                    percentages.append((float(r)/float(g))+(float(r)/float(b))+(float(g)/float(b)))

                i = 0
                pos = 0
                best = percentages[0]
                for p in percentages[1:]:
                    if p < best:
                        pos = i
                    i = i +1

                #Assign new pixel values
                im[px] = [solutions[pos][0],solutions[pos][1],solutions[pos][2],A]
            px = px+1
            
    return image

In [24]:
def new_data(image_dir,bg_dir):
    
    
    # Import background image
    background_img_raw = Image.open(bg_dir).convert('RGBA')  
    background_img_raw = background_img_raw.resize((150,150), Image.ANTIALIAS)
    background_img = np.array(background_img_raw)  
    background_img_float = background_img.astype(float)  

    # Import foreground image
    foreground_img_raw = Image.open(image_dir)  
    foreground_img = np.array(foreground_img_raw)  
    foreground_img_float = foreground_img.astype(float)  

    # Blend images
    opacity = 1  
    blended_img_float = blend_modes.grain_merge(background_img_float, foreground_img_float, opacity)

    # Convert blended image back into PIL image
    blended_img = np.uint8(blended_img_float)
    blended_img_raw = Image.fromarray(blended_img)  
    
    foreground_img_raw = foreground_img_raw.resize((149,149), Image.ANTIALIAS)
    blended_img_raw.paste(foreground_img_raw, (0, 0), foreground_img_raw)
    blended_img_raw = blended_img_raw.resize((48,48), Image.ANTIALIAS)
    
    #temp = np.uint8(blended_img_raw)
    #temp = random_noise_method(temp)
    
    #blended_img_raw = Image.fromarray(np.uint8(temp)) 
    
    
    return blended_img_raw


In [25]:
directory = 'SGTSD/Images'
if (not os.path.exists("SGTSD/Images")):
    for sign in load_paths("Traffic_Signs_Templates/Images"):
        head,tail = sign.split('.')
        name = []
        name = head.split('/')
        os.makedirs("SGTSD/Images/"+name[-1])

In [26]:
content = '''
-----------------------------------------------
|                     -*-                     |
|Synthetically Generated Traffic Sign Dataset |
|                     -*-                     |
-----------------------------------------------

This directory contains the training set for
The Convolutional Neural Network (CNN)
Used in this project

However, it can be used for any classifier
desired by the person using the code and
additionally, it is not limited to a specific
traffic sign templates.
 

----------------------------------------------
Content
----------------------------------------------

The number of example is based on the number:
->of traffic signs that were used as templates
->of the image manipulation processes
->of the brighness variations values used
->of the blending procedures


----------------------------------------------
Image format and naming
----------------------------------------------
The images created are of "jpg" format
with RGBA channels

   SIGN_X/XXX_YYY.jpg

The initial part (X) is used to distinguish the
sign class, while the remaining (XXX_YYY) firstly
indicated the sign in the file itself and the
example number.


----------------------------------------------
Additional information
----------------------------------------------

contact email: 
    
	asterga@essex.ac.uk


----------------------------------------------
Alexandros Stergiou
"The Driver's Assistant"

University of Essex,
Schoolf of Computer Science and
Electronic Engineering,
UK
----------------------------------------------
'''
text_file = open("SGTSD/Readme_Images.txt", "w")
text_file.write(content)
text_file.close()

In [27]:
def create_paths_list(imgs_directory,bg_directory):
    directories = []
    for places in load_paths(imgs_directory):
        for imgs in load_paths(places):
            dr = []
            dr = imgs.split('/')
            bg = bg_directory +'/'+dr[-2]+'/'+dr[-1]+".png"
            for signs in load_paths(imgs):
                for png in load_paths(signs):
                    directories.append([png,bg])
    return directories
                    

In [28]:
directories = create_paths_list("Traffic_Signs_exposure_manipulation","Google_search_backgrounds")
print "Files to be generated: "+str(len(directories))

Files to be generated: 2827865


In [29]:
def list_for_sign_x(i,directories):
    l = []
    for elements in directories:
        foreground = elements[0].split('/')
        background = elements[1].split('/')
        if (foreground[-2] == ("SIGN_"+str(i))):
            l.append(elements)
    return l

In [30]:
final_directories = []
signs = load_paths('Traffic_Signs_Templates/Images')
for i in range(0,len(signs)):
    final_directories.append(list_for_sign_x(i,directories))

In [31]:
direct = "SGTSD/Images"
i = 0
for element in final_directories:
    print "Processed: "+str(float(i)/float(len(final_directories))*100)+" %"
    j = 0
    for dirs in element:
        image = new_data(dirs[0],dirs[1])
        image.save(direct+"/"+str(i)+"/"+str(i)+"_"+str(j)+".png")
        j = j+1
    i = i+1
print "Processed: "+str(100)+" %"

Processed: 0.0 %
Processed: 2.32558139535 %
Processed: 4.6511627907 %
Processed: 6.97674418605 %
Processed: 9.3023255814 %
Processed: 11.6279069767 %
Processed: 13.9534883721 %
Processed: 16.2790697674 %
Processed: 18.6046511628 %
Processed: 20.9302325581 %
Processed: 23.2558139535 %
Processed: 25.5813953488 %
Processed: 27.9069767442 %
Processed: 30.2325581395 %
Processed: 32.5581395349 %
Processed: 34.8837209302 %
Processed: 37.2093023256 %
Processed: 39.5348837209 %
Processed: 41.8604651163 %
Processed: 44.1860465116 %
Processed: 46.511627907 %
Processed: 48.8372093023 %
Processed: 51.1627906977 %
Processed: 53.488372093 %
Processed: 55.8139534884 %
Processed: 58.1395348837 %
Processed: 60.4651162791 %
Processed: 62.7906976744 %
Processed: 65.1162790698 %
Processed: 67.4418604651 %
Processed: 69.7674418605 %
Processed: 72.0930232558 %
Processed: 74.4186046512 %
Processed: 76.7441860465 %
Processed: 79.0697674419 %
Processed: 81.3953488372 %
Processed: 83.7209302326 %
Processed: 86.0

In [32]:
string = '''
-------------------------------------
BREAKDOWN OF FILES GENERATED BY CLASS
-------------------------------------
'''
total = 0
for i in range (0,len(final_directories)):
    s = "Generated "+str(len(final_directories[i]))+" examples for sign class "+str(i+1)
    string = string + '\n'+s+'\n'
    total = total + len(final_directories[i])
string = string + '\n'+"TOTAL: "+str(total)+'\n'+"Generated on: "+datetime.now().strftime("%Y-%m-%d %H:%M")+'\n'
string = string + "-------------------------------------"
text_file = open("SGTSD/generated_images_about.txt", "w")
text_file.write(string)
text_file.close()

In [33]:
def png_to_jpeg(filepath):
    dirs = filepath.split('/')
    title,extension = dirs[-1].split('.')
    del dirs[-1]
    string = '/'.join(dirs)
    string = string+'/'+title+".jpg"
    png = Image.open(filepath)
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    background.save(string, 'JPEG', quality=100)
    os.remove(filepath)

In [34]:
dirs = direct = "SGTSD/Images"
i = 1
for path in load_paths(dirs):
    print "Processed: "+str(float(i-1)/float(len(final_directories))*100)+" %"
    for image in load_paths(path):
        if (image.endswith("png")):
            png_to_jpeg(image)
    i = i+1
print "Processed: "+str(100)+" %"

Processed: 0.0 %
Processed: 2.32558139535 %
Processed: 4.6511627907 %
Processed: 6.97674418605 %
Processed: 9.3023255814 %
Processed: 11.6279069767 %
Processed: 13.9534883721 %
Processed: 16.2790697674 %
Processed: 18.6046511628 %
Processed: 20.9302325581 %
Processed: 23.2558139535 %
Processed: 25.5813953488 %
Processed: 27.9069767442 %
Processed: 30.2325581395 %
Processed: 32.5581395349 %
Processed: 34.8837209302 %
Processed: 37.2093023256 %
Processed: 39.5348837209 %
Processed: 41.8604651163 %
Processed: 44.1860465116 %
Processed: 46.511627907 %
Processed: 48.8372093023 %
Processed: 51.1627906977 %
Processed: 53.488372093 %
Processed: 55.8139534884 %
Processed: 58.1395348837 %
Processed: 60.4651162791 %
Processed: 62.7906976744 %
Processed: 65.1162790698 %
Processed: 67.4418604651 %
Processed: 69.7674418605 %
Processed: 72.0930232558 %
Processed: 74.4186046512 %
Processed: 76.7441860465 %
Processed: 79.0697674419 %
Processed: 81.3953488372 %
Processed: 83.7209302326 %
Processed: 86.0

In [35]:
shutil.rmtree("Traffic_Signs_exposure_manipulation")
shutil.rmtree("Traffic_Signs_Templates/Transformed_Images")
shutil.rmtree("Traffic_Signs_Templates/Processed_Images")